In [ ]:
import cognify

In [ ]:
#================================================================
# Evaluator
#================================================================

metric = cognify.metric.f1_score_str

def evaluate_answer(answer, label):
    return metric(answer, label)

In [ ]:
#================================================================
# Data Loader
#================================================================

import json

def load_data_minor():
    with open("data._json", "r") as f:
        data = json.load(f)
          
    # format to (input, output) pairs
    new_data = []
    for d in data:
        input = {
            'question': d["question"], 
            'documents': d["docs"]
        }
        output = {
            'label': d["label"],
        }
        new_data.append((input, output))
    return new_data[:5], None, new_data[5:]

In [ ]:
#================================================================
# Optimizer Set Up
#================================================================
from cognify.hub.search import default
search_settings = default.create_search()

In [ ]:
train, val, test = load_data_minor()

In [ ]:
opt_cost, pareto_frontier, opt_logs = cognify.optimize(
    script_path="workflow.py",
    control_param=search_settings,
    train_set=train,
    val_set=val,
    eval_fn=evaluate_answer,
    force=True,
)

In [ ]:
new_workflow = cognify.load_workflow(config_id='Pareto_1', opt_result_path='opt_results')

In [ ]:
question = "What was the 2010 population of the birthplace of Gerard Piel?"
documents = [
    'Gerard Piel | Gerard Piel (1 March 1915 in Woodmere, N.Y. – 5 September 2004) was the publisher of the new Scientific American magazine starting in 1948. He wrote for magazines, including "The Nation", and published books on science for the general public. In 1990, Piel was presented with the "In Praise of Reason" award by the Committee for Skeptical Inquiry (CSICOP).',
]

new_workflow(question=question, documents=documents)

In [ ]:
eval_result = cognify.evaluate(
    config_id='Pareto_1',
    test_set=test,
    opt_result_path='opt_results',
    n_parallel=10,
    eval_fn=evaluate_answer,
)

In [ ]:
cognify.inspect(
    'opt_results'
)

In [ ]:
eval_result = cognify.evaluate(
    config_id='NoChange',
    test_set=test,
    workflow='workflow.py',
    n_parallel=10,
    eval_fn=evaluate_answer,
)